# DTW with recorded coordinates

In [2]:
pip install fastdtw

     -------------------------------------- 133.4/133.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3566 sha256=8dc73635b1d17f45f690722307b22a0dd5be93f785ce90962cdff3d5a28e75ba
  Stored in directory: c:\users\leesoohwan\appdata\local\pip\cache\wheels\e9\ac\30\c962f9d759dd68cb5482727c44441fdfb48040fdbe983857e8
Successfully built fastdtw
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

# Multi-person Pose Estimation with MoveNet

In [3]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Load Video
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

# Compare each coordinate with DTW

In [ ]:
import numpy as np
import cv2
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# assume video_path is the path to the video file
# # video_path = "path/to/video.mp4"
# video_path = '../../data/videos/'
# sampleVideo = video_path + 'sampleVideo.mp4'
# basicFour = video_path + 'basic_four.mp4'
# basicTwo = video_path + 'basic_two.mp4'
# swingDiff = video_path + 'swing_diff.mp4'
# swingSame = video_path + 'swing_same.mp4'
# swingRaw = video_path + 'swingRaw.mp4'
# temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
# temporalEasy = video_path + 'temporalEasyRaw.mp4'

# assume pose1_coords is a numpy array of shape (M, 2),
# where M is the number of pose keypoints * 2
pose1_coords = None

# read the video using OpenCV
cap = cv2.VideoCapture(swingSame)

while cap.isOpened():
    # read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break

    # # assume pose estimation code extracts the pose coordinates from the frame
    # pose2 = pose_estimation(frame)

    if pose1_coords is None:
        # initialize the reference pose sequence with the first pose
        pose1_coords = pose2.flatten()
        continue

    # compute the Euclidean distance between corresponding keypoints
    distance = lambda a, b: euclidean(a, b)

    # compute the DTW distance and path
    distance, path = fastdtw(pose1_coords.reshape(1, -1), pose2.flatten().reshape(1, -1), dist=distance)

    # compute the similarity between the poses (using a normalized distance)
    similarity = 1.0 / (1.0 + distance)

    # compute the time difference between the current frame and the reference frame
    time_diff = (path[-1][0] - path[0][0]) / float(len(pose1_coords))

    # update the reference pose sequence with the current pose
    pose1_coords = pose2.flatten()

    # print the results
    print("Pose similarity: ", similarity)
    print("Time difference: ", time_diff)